In [ ]:
import utils
import emission
import transition

In [ ]:
LOWER = True
NORM_TENSE = True
REP_NUM = True
REP_YEAR = True
REP_SYM = False

In [3]:
dataset_folder = "data/EN/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

In [4]:
emission_data = emission.generate_emission_table(lines,
                                                 lower=LOWER,
                                                 norm_tense=NORM_TENSE,
                                                 replace_number=REP_NUM,
                                                 replace_year=REP_YEAR,
                                                 replace_symbol=REP_SYM)
hashmap = emission_data["x_hashmap"]
word_freq = emission_data["x_word_freq"]
smoothed_hashmap = utils.add_unk(hashmap, word_freq, k=4)
emission_data["x_hashmap"] = smoothed_hashmap

x_vocab = utils.get_emission_vocab(smoothed_hashmap)
print("Vocab size:", len(x_vocab))

not enough values to unpack (expected 2, got 1)
Skipped 1 lines:  ['']
Vocab size: 4216


In [5]:
transition_pairs = transition.generate_transition_pairs(lines)

In [6]:
y_pairs = transition_pairs["Y_pairs"]
y_vocab = transition_pairs["y_vocab"]
y_freq = transition_pairs["y_freq"]

In [7]:
transition_data = transition.generate_transition_data(y_pairs, y_vocab)

In [8]:
import viterbi

In [9]:
hmm = viterbi.HMM()
hmm.fit_word_tokenizer(x_vocab)
hmm.fit_pos_tokenizer(y_vocab)
hmm.build_transition_weights(y_freq, transition_data)
hmm.build_emission_weights(emission_data)

In [10]:
train_data = dataset_folder + "dev.in"
lines = utils.read_file_to_lines(train_data)

sentences = []

while len(lines) > 1:
    sentence_break = lines.index("")
    sentence_xy = lines[:sentence_break]
    words = [utils.preprocess_text(token,
                                   lower=LOWER,
                                   norm_tense=NORM_TENSE,
                                   replace_number=REP_NUM,
                                   replace_year=REP_YEAR,
                                   replace_symbol=REP_SYM)
             for token in sentence_xy]
    sentence = " ".join(words).strip()
    sentences.append(sentence)
    lines = lines[sentence_break+1:]

In [11]:
sentences[100:110]

['the senate estimate that its version of the provision would yield $ 17 million the first year and a total of $ 409 million over five year .',
 'trinova third-quarter loss after a charge for a planned restructuring , which will include the closing or downsizing of about 25 % of its plant and a work force cut of about 1500 over three year .',
 'dealer said it is hit by some profit-taking after gain since mid-week .',
 'gm said the assembly plant , which build the chevrolet corsica and beretta compact cars , originally is scheduled to reopen monday but now will not resume production until oct. 30 .',
 "the lawsuit state that unless the sanctions are halted pending an appeal , the broker and his firm `` will be irreparably injured and their business will be totally and permanently destroyed . ''",
 'construction of the NUM-floor building will begin next may and should be completed in april 1992 .',
 "mr. batchelder says that in the past , bank would normally have loaned 65 % of a total b

In [12]:
# only for the progress bar!
try:
    from tqdm import tqdm
    USE_TQDM = True
except Exception as e:
    print(e, "TQDM import error, disable progress bar")

if USE_TQDM:
    sentences_it = tqdm(sentences)
else:
    sentences_it = sentences

  0%|          | 0/1094 [00:00<?, ?it/s]

In [13]:
preds = []

for line in sentences_it:
    pred = hmm.viterbi_predict(line)
    pred = hmm.pos_tokens_to_labels(pred)
    preds.append(pred)
    
assert len(sentences) == len(preds)

100%|██████████| 1094/1094 [00:32<00:00, 33.67it/s]


In [14]:
outfile = dataset_folder + "dev.p5.out"

with open(outfile, "w") as f:
    for sentence, pred in zip(sentences, preds):
        word_array = sentence.split(" ")
        try:
            assert len(word_array) == len(pred)
            for i, word in enumerate(word_array):
                f.write(word + " " + pred[i] +"\n")
        except:
            print(word_array)
            print(pred)
            break
        f.write("\n")

In [15]:
!python3 evalResult.py ./data/EN/dev.out ./data/EN/dev.p5.out


#Entity in gold data: 13179
#Entity in prediction: 12913

#Correct Entity : 10936
Entity  precision: 0.8469
Entity  recall: 0.8298
Entity  F: 0.8383

#Correct Sentiment : 10543
Sentiment  precision: 0.8165
Sentiment  recall: 0.8000
Sentiment  F: 0.8081
